In [1]:
import urllib.request
import urllib.parse
import json
import regex as re
import pandas as pd
from tqdm import tqdm
import os

In [2]:
# List of disney movies
# wiki_url = https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films

baseurl = "https://en.wikipedia.org/w/api.php?"
action = "action=query"
title = "titles=List_of_Walt_Disney_Pictures_films"
content = "prop=revisions&rvprop=content"
dataformat ="format=json"
rvslots = "rvslots=main"

query = "{}{}&{}&{}&{}&{}".format(baseurl, action, content, title, dataformat, rvslots)

headers = {"User-Agent" : "MyWikipediaClient/1.0 (example@example.com)"} # just use this dict as-is.
wikirequest = urllib.request.Request(query,None,headers)    # Needed to pass error 403
wikiresponse = urllib.request.urlopen(wikirequest)
wikidata = wikiresponse.read()
wikitext = wikidata.decode('utf-8')

In [3]:
# Load JSON and extract page content
wiki_json = json.loads(wikitext)
pages = wiki_json['query']['pages']['1970335']['revisions'][0]['slots']['main']['*']

In [4]:
# Example of a movie name listed -- {{Date table sorting|March 10, 2000}}\n| \'\'[[Whispers: An Elephant\'s Tale]]\'\'\n|\n|-\n|
# Build regex to extract all movie names
pattern = r"Date table sorting[^}]+}}\n\|\s*''?\[\[([^\]|]+?)(?:\|[^\]]+)?\]\]"
movies = re.findall(pattern, pages)

In [5]:
# Count opening and closing braces to find the true end (generated by an LLM)
def extract_infobox_manual(text):
    """Extract infobox by counting nested braces"""
    start = text.find('{{Infobox')
    if start == -1:
        return None
    
    count = 0
    i = start
    while i < len(text):
        if text[i:i+2] == '{{':
            count += 1
            i += 2
        elif text[i:i+2] == '}}':
            count -= 1
            i += 2
            if count == 0:
                return text[start:i]
        else:
            i += 1
    return None

In [6]:
# Extract infobox and plot for each movie
for name in tqdm(movies):

    # Properly encode the title to handle special characters like :, -, etc.
    encoded_title = urllib.parse.quote(name, safe='')
    movie_name = f"titles={encoded_title}"

    query = "{}{}&{}&{}&{}&{}".format(baseurl, action, content, movie_name, dataformat, rvslots)

    try:
        movie_wikirequest = urllib.request.Request(query, None, headers)    # Needed to pass error 403
        movie_wikiresponse = urllib.request.urlopen(movie_wikirequest)
        movie_wikidata = movie_wikiresponse.read()
        movie_wikitext = movie_wikidata.decode('utf-8')

        movie_wikijson = json.loads(movie_wikitext)
        movie_pageid = list(movie_wikijson['query']['pages'].keys())[0]
        movie_pagecontent = movie_wikijson['query']['pages'][movie_pageid]['revisions'][0]['slots']['main']['*']
        
        # Separate extraction of infobox and plot to avoid unwanted content

        # Regex pattern matching for extracting plot
        plot_pattern = r'==\s*Plot\s*==.*?(?=\n==(?!=))'
        plot_match = re.search(plot_pattern, movie_pagecontent, re.DOTALL)

        # Extract infobox by matching brackets since every infobox is not the same
        infobox_content = extract_infobox_manual(movie_pagecontent)

        # Save them together if they both exist
        if infobox_content and plot_match:

            # Get clean plot content
            plot_content = plot_match.group(0)

            # Combine infobox and plot contents
            combined_content = infobox_content + "\n\n" + plot_content

            # Clean movie name for file saving
            clean_movie_name = name.replace('_', ' ')

            # Save to a text file
            with open(f"movies\{clean_movie_name}.txt", "w", encoding='utf-8') as f:
                f.write(combined_content)
        else:
            if not infobox_content:
                print(f"Infobox not found for {name}.")
            if not plot_match:
                print(f"Plot not found for {name}.")

    except Exception as e:
        print(f"Error retrieving data for {name}: {e}.")

  1%|          | 4/516 [00:04<08:11,  1.04it/s]

Plot not found for The Reluctant Dragon (1941 film).


  1%|▏         | 7/516 [00:06<07:17,  1.16it/s]

Plot not found for Saludos Amigos.


  2%|▏         | 8/516 [00:07<07:04,  1.20it/s]

...

 96%|█████████▋| 497/516 [07:44<00:17,  1.07it/s]

Infobox not found for World's Best (film).
Plot not found for World's Best (film).


 98%|█████████▊| 504/516 [07:50<00:10,  1.14it/s]

Plot not found for The Beach Boys (film).


100%|██████████| 516/516 [08:02<00:00,  1.07it/s]


In [8]:
len(os.listdir('movies'))

469